# Домашнее задание к лекции "Основы веб-скрапинга и работы с API"

## Задание 1. 

### Обязательная часть

Будем парсить страницу со свежеми новостям на [habr.com/ru/all/](https://habr.com/ru/all/).

Вам необходимо собирать только те статьи, в которых встречается хотя бы одно требуемое ключевое слово. Эти слова определяем в начале кода в переменной, например:

`KEYWORDS = ['python', 'парсинг']`

 Поиск вести по всей доступной preview-информации (это информация, доступная непосредственно с текущей страницы). 
 
В итоге должен формироваться датафрейм вида: `<дата> - <заголовок> - <ссылка>`

### Дополнительная часть (необязательная)

Улучшить скрипт так, чтобы он анализировал не только preview-информацию статьи, но и весь текст статьи целиком.

Для этого потребуется получать страницы статей и искать по тексту внутри этой страницы.  

Итоговый датафрейм формировать со столбцами: `<дата> - <заголовок> - <ссылка> - <текст_статьи>`


In [10]:
import requests
import pandas as pd

In [11]:
from bs4 import BeautifulSoup

In [100]:
url = 'https://habr.com/ru/all/'
res = requests.get(url)
soup = BeautifulSoup(res.text, 'html.parser')

In [162]:
KEYWORDS = ['signal', 'java']

In [163]:
# извлекаем посты
posts = soup.find_all('article', class_='post')

In [178]:
my_data = []
my_data_full = []

# добавляем извлечение хабов из постов, чтобы отбирать только нужные
for post in posts:
    post_id = post.parent.attrs.get('id')
   # если идентификатор не найден, это что-то странное, пропускаем
    if not post_id:
        continue
    
    # первая часть задания
    date_ = post.find('span', class_='post__time')
    link_all = post.find('a', class_='post__title_link')
    link = link_all.attrs.get('href')
    
    if any([desired in link_all.text.lower() for desired in KEYWORDS]): 
        to_add = [date_.text, link_all.text, link]
        my_data += [to_add]
         
    # вторая часть задания
    res_full = requests.get(link)
    soup2 = BeautifulSoup(res_full.text, 'html.parser')
    full_text = soup2.find('div', class_='post__body post__body_full')
    
    if any([desired in full_text.text.lower() for desired in KEYWORDS]): 
        to_add_full = [date_.text, link_all.text, link, full_text.text]
        my_data_full += [to_add_full]

In [179]:
df_columns = ['data', 'title', 'link']
df = pd.DataFrame(my_data, columns=df_columns)

In [180]:
df

,data,title,link
0,сегодня в 00:33,Signal: Взлом Cellebrite с атакованного устрой...,https://habr.com/ru/post/554188/
1,вчера в 17:10,"API, ради которых наконец-то стоит обновиться ...",https://habr.com/ru/post/554128/


In [182]:
df_columns = ['data', 'title', 'link','full_text']
df_full = pd.DataFrame(my_data_full, columns=df_columns)

In [183]:
df_full

,data,title,link,full_text
0,сегодня в 08:17,Русские программисты не сдаются-2,https://habr.com/ru/post/554104/,\n\n\n «Главное в следственных действиях – \r\...
1,сегодня в 00:33,Signal: Взлом Cellebrite с атакованного устрой...,https://habr.com/ru/post/554188/,\nКомпания Cellebrite производит программное о...
2,вчера в 17:10,"API, ради которых наконец-то стоит обновиться ...",https://habr.com/ru/post/554128/,\nКакие есть причины переходить на новые верси...
3,вчера в 15:15,Лучшие экспериментальные протоколы для исследо...,https://habr.com/ru/company/skillfactory/blog/...,\nЗолотым стандартом исследований в области ма...
4,вчера в 14:01,Релиз Node.js 16: обзор лучшего,https://habr.com/ru/company/macloud/blog/554098/,\n\n\r\n20 апреля состоялся релиз новой 16 вер...


## Задание 2.

### Обязательная часть

Написать скрипт, который будет проверять список e-mail адресов на утечку при помощи сервиса [Avast Hack Ckeck](https://www.avast.com/hackcheck/).
Список email-ов задаем переменной в начале кода:  
`EMAIL = [xxx@x.ru, yyy@y.com]`

В итоге должен формироваться датафрейм со столбцами: `<почта> - <дата утечки> - <источник утечки> - <описание утечки>`  

**Подсказка**: сервис работает при помощи "скрытого" API. Внимательно изучите post-запросы.

In [2]:
URL = 'https://identityprotection.avast.com/v1/web/query/site-breaches/unauthorized-data'

In [3]:
headers = {
    'Accept': 'application/json, text/plain, */*',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7',
    'Connection': 'keep-alive',
    'Content-Length': '46',
    'Content-Type': 'application/json;charset=UTF-8',
    'Host': 'identityprotection.avast.com',
    'Origin': 'https://www.avast.com',
    'Referer': 'https://www.avast.com/',
    'sec-ch-ua': '"Google Chrome";v="89", "Chromium";v="89", ";Not A Brand";v="99"',
    'sec-ch-ua-mobile': '?0',
    'Sec-Fetch-Dest': 'empty',
    'Sec-Fetch-Mode': 'cors',
    'Sec-Fetch-Site': 'same-site',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.128 Safari/537.36',
    'Vaar-Header-App-Build-Version': '1.0.0',
    'Vaar-Header-App-Product': 'hackcheck-web-avast',
    'Vaar-Header-App-Product-Name': 'hackcheck-web-avast',
    'Vaar-Version': '0'
}

In [4]:
res = requests.post(URL, headers = headers, json = {"emailAddresses":["martynovpeter@gmail.com","peter.box@yandex.ru"]})
res

<Response [200]>

In [5]:
result = res.json()

In [22]:
list1 = []
for key, value in result['data'].items():
    
    email = list(value.keys())[0]
    source = key
    breachId = str(list(value.values())[0][0]['breachId'])
    list1 += [[breachId,email,source]]

In [23]:
df1 = pd.DataFrame(list1, columns = ['breachId','email','source'])

In [24]:
df1

,breachId,email,source
0,16372,martynovpeter@gmail.com,bookmate.com
1,16003,peter.box@yandex.ru,egpet.net
2,41,peter.box@yandex.ru,dropbox.com
3,16768,martynovpeter@gmail.com,canva.com


In [25]:
list1 = []
for key, value in result['breaches'].items():
    breachId = str(key)
    date = value['publishDate']
    description = value['description']
    list1 += [[breachId, date, description]]
    

In [26]:
df2 = pd.DataFrame(list1, columns = ['breachId','date','description'])

In [27]:
df2

,breachId,date,description
0,16372,2019-03-20T00:00:00Z,"In July 2018, Bookmate was allegedly breached...."
1,16003,2018-08-15T00:00:00Z,"In September 2017, Petroleum Community Forum's..."
2,16768,2019-06-13T00:00:00Z,"In May 2019, graphic-design site Canva's datab..."
3,41,2016-10-24T00:00:00Z,Cloud storage company Dropbox suffered a major...


In [29]:
df1 = df1.merge(df2, left_on='breachId', right_on='breachId')

In [31]:
df1.drop(columns='breachId', axis = 1, inplace = True)

In [33]:
columnsTitles=['email','date','source','description']
df1=df1.reindex(columns=columnsTitles)

In [34]:
df1

,email,date,source,description
0,martynovpeter@gmail.com,2019-03-20T00:00:00Z,bookmate.com,"In July 2018, Bookmate was allegedly breached...."
1,peter.box@yandex.ru,2018-08-15T00:00:00Z,egpet.net,"In September 2017, Petroleum Community Forum's..."
2,peter.box@yandex.ru,2016-10-24T00:00:00Z,dropbox.com,Cloud storage company Dropbox suffered a major...
3,martynovpeter@gmail.com,2019-06-13T00:00:00Z,canva.com,"In May 2019, graphic-design site Canva's datab..."


### Дополнительная часть (необязательная)

Написать скрипт, который будет получать 50 последних постов указанной группы во Вконтакте.  
Документация к API VK: https://vk.com/dev/methods
, вам поможет метод [wall.get](https://vk.com/dev/wall.get)  
```
GROUP = 'netology'  
TOKEN = УДАЛЯЙТЕ В ВЕРСИИ ДЛЯ ПРОВЕРКИ, НА GITHUB НЕ ВЫКЛАДЫВАТЬ  
```

В итоге должен формироваться датафрейм со столбцами: `<дата поста> - <текст поста>`

In [71]:
from datetime import datetime

In [6]:
TOKEN = ''

In [15]:
GROUP = 'netology' 
VERSION = '5.58'
REQUEST = 'https://api.vk.com/method/groups.getById'

In [16]:
params = {
    'access_token': TOKEN,
    'v': VERSION,
    'group_id': GROUP
}

In [27]:
res = requests.get(REQUEST, params)
res

<Response [200]>

In [28]:
res = res.json()
group_id = res['response'][0]['id']
group_id

30159897

In [31]:
REQUEST = 'https://api.vk.com/method/wall.get'

In [55]:
params = {
    'access_token': TOKEN,
    'v': VERSION,
    'owner_id': -group_id,
    'count': 50
}

In [63]:
res = requests.get(REQUEST, params)
res

<Response [200]>

In [64]:
res = res.json()

In [66]:
my_results = res['response']['items']

In [77]:
df = pd.DataFrame(my_results)

In [78]:
df = df.loc[:, ['date','text']]

In [80]:
df['date'] = df.apply(lambda x: datetime.utcfromtimestamp(x['date']).strftime('%Y-%m-%d %H:%M:%S'), axis=1)

In [81]:
df

,date,text
0,2021-04-14 13:20:03,А у нас конкурс 🎁\n\nРазыгрываем сразу 10 приз...
1,2021-04-24 09:37:00,"Управленческие навыки — важные софт-скиллы, ко..."
2,2021-04-23 14:08:00,"Чем бы вы ни занимались, важно уметь мыслить к..."
3,2021-04-23 07:37:00,"Начать карьеру в дизайне просто, для старта — ..."
4,2021-04-22 15:26:00,Научиться продвигать коммерческий профиль без ...
5,2021-04-22 08:03:00,"Макака играет в видеоигру силой мысли, роботы ..."
6,2021-04-21 13:39:00,Уже 10 лет мы в Нетологии обучаем диджитал-про...
7,2021-04-21 07:22:00,👉 26 апреля стартует бесплатный курс «Python-р...
8,2021-04-20 15:20:00,«Я никогда не...» 🤔 \n \nНапоминаем об основны...
9,2021-04-20 07:55:00,"Видели фото 360⁰ бытовой техники на М.Видео, п..."
